<a href="https://colab.research.google.com/github/Roopasree-Pachipulusu/-Roopasree-Pachipulusu-.github.io/blob/main/Identification_of_cotton_species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install Requirements

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12396, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 12396 (delta 4), reused 8 (delta 1), pack-reused 12380
Receiving objects: 100% (12396/12396), 12.17 MiB | 28.78 MiB/s, done.
Resolving deltas: 100% (8547/8547), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.1 MB/s 
     |████████████████████████████████| 145 kB 6.0 MB/s 
     |████████████████████████████████| 178 kB 40.5 MB/s 
     |████████████████████████████████| 3.1 MB 42.9 MB/s 
     |████████████████████████████████| 1.1 MB 42.7 MB/s 
     |████████████████████████████████| 21.8 MB 1.4 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 138 kB 66.6 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account

# Step 2: Assemble Our Dataset

In order to train our custom model, we have assembeled our cotton image dataset of representative images with bounding box annotations around the objects that we want to detect. 
In this part, we convert annotations into the format expected by YOLO v5.YOLO v5 expects annotations for each image in form of a .txt file where each line of the text file describes a bounding box.
We have used Roboflow to Annotate and Assemble the dataset 



# Annotate

![picture](https://drive.google.com/uc?id=1-0JJY8bbFO6FQoX-a2x4PWFCSAazu_Tp)





# Step 3: Assemble Our Dataset

# Pre-processing and Data Agumentation 

![picture](https://drive.google.com/uc?id=1mNcCPNIupuiJxo7qLUaemmCALZ8lw-9d)

# Step 4: Assemble Our Dataset
#Partitioning the data set
Next we partition the dataset into train, validation, and test sets containing 80%, 10%, and 10% of the data, respectively. 


* train - It is the set of data that is used to train and make the model learn the hidden features/patterns in the data.

* validation -The validation set is a set of data, separate from the training set, that is used to validate our model performance during training.
This validation process gives information that helps us tune the model’s hyperparameters and configurations accordingly. It is like a critic telling us whether the training is moving in the right direction or not.
The model is trained on the training set, and, simultaneously, the model evaluation is performed on the validation set after every epoch.
The main idea of splitting the dataset into a validation set is to prevent our model from overfitting i.e., the model becomes really good at classifying the samples in the training set but cannot generalize and make accurate classifications on the data it has not seen before. 

* test - The test set is a separate set of data used to test the model after completing the training.
![picture](https://drive.google.com/uc?id=1GTe-GAXIptegmUMUyz3bR3ajaEeEWOeh)



In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/drive/MyDrive/datasets"

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ZYZwQ6u1cXtNANHiAdAS")
project = rf.workspace("cotton-nqp2x").project("bt-cotton")
dataset = project.version(5).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-8.4.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/drive/MyDrive/datasets/bt-cotton-5 in yolov5pytorch:: 100%|██████████| 4258/4258 [15:21<00:00,  4.62it/s]


# Step 5: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training
- **hyp**  YAML file that describes hyperparameter choices. For examples of how to define hyperparameters, see data/hyp.scratch.yaml. If unspecified, the file data/hyp.scratch.yaml is used
-**Data Config File** 
Details for the dataset we want to train your model on are defined by the data config YAML file. The following parameters have to be defined in a data config file:
  1.   train, test, and val: Locations of train, test, and validation images.
  2. nc: Number of classes in the dataset. 
  3. names: Names of the classes in the dataset. The index of the classes in this list would be used as an identifier for the class names in the code.



In [ ]:
# !python train.py --img 416 --batch 16 --epochs 100 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results --cache --project '../gdrive/MyDrive/' --name 'backup'

In [ ]:
!python train.py --img 416 --batch 20 --epochs 50 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache 

train: weights=yolov5s.pt, cfg=, data={dataset.location}/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=20, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/, name=backup_weights, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
Traceback (most recent call last):
  File "train.py", line 666, in <module>
    main(opt)
  File "train.py", line 536, in main
    check_file(opt.data), check_yaml(opt.cfg), check_yaml(opt.hyp), str(opt.weights), str(opt.project)  # checks
  File "/content/yolov5/yolov5/utils/general.py", line 435, in check_file
  

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source /content/drive/MyDrive/datasets/bt-cotton-5/test/images

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/datasets/bt-cotton-5/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-253-g75bbaa8 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients
image 1/74 /content/drive/MyDrive/datasets/bt-cotton-5/test/images/a-115_jpeg.rf.46e69dc3ffc91d96a9c54926095fd10a.jpg: 416x416 1 G.herbaceum, Done. (0.011s)
image 2/74 /content/drive/MyDrive/datasets/bt-cotton-5/test/images/a-115_jpeg.rf.ae24b4f82f5873356871cf2546450b33.jpg: 416x416 Done. (0.011s)
image 3/74 /content/drive/MyDrive/datas

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob(''): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Conclusion and Further enhancement



To improve you model's performance, we recommend first interating on your datasets coverage and quality. See this guide for [model performance improvement](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results).

To deploy your model to an application, see this guide on [exporting your model to deployment destinations](https://github.com/ultralytics/yolov5/issues/251).

Once your model is in production, you will want to continually iterate and improve on your dataset and model via [active learning](https://blog.roboflow.com/what-is-active-learning/).

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>